2019 Census Data for Selected Variables - Baltimore City

In [1]:
# STILL NEED TO CALCULATE THE FOLLOWING STATS:
# HOME OWNERSHIP RATE, HH DENSITY, HH GROWTH, INCOME GROWTH, POP DENSITY, POP GROWTH

In [2]:
#pip install us

In [3]:
# pip install censusgeocode
# pip install censusdata



In [4]:
# pip install git+git://github.com/geopandas/geopandas.git

In [5]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [6]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
import censusdata 
import censusgeocode as cg
#import gmaps

# Census & gmaps API Keys
from config import api_key
c = Census(api_key, year=2019)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [7]:
# testing search function
#sample = censusdata.search('acs5', 2019, 'concept', 'Land')
#print(sample)


[('B01001E_001E', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)', 'Estimate!!Total:'), ('B01001E_002E', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)', 'Estimate!!Total:!!Male:'), ('B01001E_003E', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)', 'Estimate!!Total:!!Male:!!Under 5 years'), ('B01001E_004E', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)', 'Estimate!!Total:!!Male:!!5 to 9 years'), ('B01001E_005E', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)', 'Estimate!!Total:!!Male:!!10 to 14 years'), ('B01001E_006E', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)', 'Estimate!!Total:!!Male:!!15 to 17 years'), ('B01001E_007E', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)', 'Estimate!!Total:!!Male:!!18 and 19 years'), ('B01001E_008E', 'SEX BY AGE (NATIVE HAWAIIAN AND OTHER PACIFIC ISLANDER ALONE)', 'Estimate!!Total:!!Male:!!20 to 24 years'), ('B01001E_009E', 'SEX BY AGE (NA

In [ ]:
# See list of all tables in the ACS5
#c.acs5.tables()

In [ ]:
#censusdata.printtable(censusdata.censustable('acs1', 2019, 'B08604'))

[('B04004_057E',
  'PEOPLE REPORTING SINGLE ANCESTRY',
  'Estimate!!Total:!!New Zealander'),
 ('B04005_057E',
  'PEOPLE REPORTING MULTIPLE ANCESTRY',
  'Estimate!!Total:!!New Zealander'),
 ('B04006_057E',
  'PEOPLE REPORTING ANCESTRY',
  'Estimate!!Total:!!New Zealander'),
 ('B05006_123E',
  'PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION IN THE UNITED STATES',
  'Estimate!!Total:!!Oceania:!!Australia and New Zealand Subregion:'),
 ('B05006_124E',
  'PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION IN THE UNITED STATES',
  'Estimate!!Total:!!Oceania:!!Australia and New Zealand Subregion:!!Australia'),
 ('B05006_125E',
  'PLACE OF BIRTH FOR THE FOREIGN-BORN POPULATION IN THE UNITED STATES',
  'Estimate!!Total:!!Oceania:!!Australia and New Zealand Subregion:!!Other Australian and New Zealand Subregion')]

In [10]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",                                                            
                          "B15003_017E",
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_008E",
                          "B03001_003E",
                          "B25008_002E",
                          "B28007_001E",
                          "B25003_002E",               
                          "B25003_003E"),               
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*")
census_pd = pd.DataFrame(census_data)
census_pd.head()
                         

CensusException: error: error: unknown variable 'AREALAND'

In [ ]:
# did not add in daytime population - will use ESRI business analyst for this 
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state_county(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          county_fips = "510") 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

In [ ]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract #
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census_pd["GEOID"] = census_pd['state'] + census_pd['county'] + census_pd['tract']
census_pd.head()

In [ ]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

In [ ]:
# Column Reordering
census_pd = census_pd.rename(columns={"B19013_001E": "Median household income",
                                      "B01003_001E": "Population",
                                      "B01002_001E": "Median age",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_008E": "Pop. 2 or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "B28007_001E": "Labor force status by presence of computer/ internet in HH",
                                      "NAME": "Name", "state": "State", "GEOID": "GEOID"})

# Add in poverty rate (Poverty Count / Population)
census_pd["Poverty rate"] = 100 * \
    census_pd["Poverty count"].astype(
        int) / census_pd["Population"].astype(int)


# Add in unemployment rate (Employment Count / Population)
census_pd["Unemployment rate"] = 100 * \
    census_pd["Unemployment count"].astype(
        int) / census_pd["Population"].astype(int)
census_pd.head()

In [ ]:
# Add in home ownership rate (# owner-occupied units / # of occupied housing units)
# sum 2 columns: total owner-occupied units + total renter-occupied units to create additional column "Total occupied units" 
sum_column = census_pd['Total owner-occupied units'] + census_pd['Total renter-occupied units']
census_pd["Total occupied units"] = sum_column

In [ ]:
census_pd["Home ownership rate"] = 100 * \
    census_pd["Total owner-occupied units"].astype(
        int) / census_pd["Total occupied units"].astype(
        int) 

census_pd.head()

In [ ]:
census_pd["Home ownership rate"] = census_pd["Home ownership rate"].astype(float).round(1)


In [ ]:
census_pd["Poverty rate"] = census_pd["Poverty rate"].astype(float).round(1)


In [ ]:
census_pd["Unemployment rate"] = census_pd["Unemployment rate"].astype(float).round(1)
census_pd.head()

In [ ]:
# Calculate population density  see: https://www.census.gov/quickfacts/fact/note/US/LND110210
# density is expressed as "population per square mile(kilometer)"
# Divide total population (or # of housing units)/ by land area of the entity measured in square miles

In [ ]:
# Change order of columns in DataFrame
census_pd = census_pd[["State", "Name", "GEOID", "Population", "Median household income",
                       "Per capita income", "Poverty count", "Poverty rate", "Unemployment rate", 
                       "# employed, age 16+", "Unemployment count",
                      "# persons age 25+ graduated high school", "# persons age 25+ with Bachelor's degree",
                      "Median age","Pop. white", "Pop. Black", "Pop. 2 or more races", "Pop. Hispanic origin", 
                      "Total pop. in occupied housing units by tenure", "Total owner-occupied units", "Total renter-occupied units",
                      "Labor force status by presence of computer/ internet in HH"]]

census_pd.head()

In [ ]:
census_pd.count()

In [ ]:

census_pd[['Census_Tract', "County", "State Name"]]= census_pd['Name'].str.split(",", n=3, expand=True)
census_pd.head()

In [ ]:
census_pd = census_pd.drop(["State"], axis=1)
census_pd.head()

In [ ]:
census_pd.columns

In [ ]:
# calculate daytime population; from: https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# Two methods for calculating commuter-adjusted population estimates:

#Method 1: Total resident population + (total workers working in area - workers who lived and worked in same area) - (total workers living in area - workers who lived and worked in same area) OR;
#Method 2: Total resident population + Total workers working in area - Total workers living in area.




In [ ]:
census_pd["Census_Tract"] = census_pd['Census_Tract'].str.replace('Census Tract', "") 
census_pd

In [ ]:
census_pd = census_pd.drop(["State Name"], axis=1)
census_pd.head()

In [ ]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_pd["Census_Tract"].unique())
tract_count

In [ ]:
census_final = census_pd.drop(["Name"], axis=1)
census_final.head()

In [ ]:
# Calculations can also be performed on Series and added into DataFrames as new columns
#thousands_of_dollars = data_file_df["Amount"]/1000
#data_file_df["Thousands of Dollars"] = thousands_of_dollars

#data_file_df.head()

In [ ]:
# Converting the membership days into weeks and then adding a column to the DataFrame
#weeks = training_df["Membership (Days)"]/7
#training_df["Membership (Weeks)"] = weeks

#training_df.head()

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
#census_pd.to_csv("Comm_Corridor_Stats_Draft2.csv", index = False, header=True)